# Задание

Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Разделите выборку на обучающую/проверочную в соотношении 80/20.

Постройте ансамбль решающих деревьев, используя градиентный бустинг (GradientBoostingClassifier). Используйте перекрестную проверку, чтобы найти наилучшие параметры ансамбля, или используйте параметры от случайного леса: max_depth=17, max_features=27, min_samples_leaf=20, n_estimators=76.

Проведите предсказание и проверьте качество через каппа-метрику.

Данные: video.ittensive.com/machine-learning/prudential/train.csv.gz

In [1]:
GRAIN = 11
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from etl_utils import reduce_mem_usage


data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")

data['Product_Info_2_1'] = data['Product_Info_2'].str.slice(0, 1)
data['Product_Info_2_2'] = pd.to_numeric(data['Product_Info_2'].str.slice(1, 2))
data = data.drop('Product_Info_2', axis='columns')

onehot_df = pd.get_dummies(data['Product_Info_2_1'])
onehot_df.columns = ['Product_Info_2_1' + column for column in onehot_df.columns]
data = pd.merge(left=data, right=onehot_df, left_index=True, right_index=True).drop('Product_Info_2_1', axis=1).fillna(-1)
del onehot_df

data['Response'] = data['Response'] - 1

### Набор столбцов для расчета

In [2]:
columns_groups = ['Insurance_History', 'InsurеdInfo', 'Medical_Keyword', 'Family_Hist', 'Medical_History', 'Product_Info']
columns = ['Wt', 'Ht', 'Ins_Age', 'BMI']
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print(columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных и Разделение данных

In [3]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(data[columns]))
columns_transformed = data_transformed.columns
data_transformed['Response'] = data['Response']
data_transformed = reduce_mem_usage(data_transformed)

data_train, data_test = train_test_split(data_transformed, test_size=.2, random_state=GRAIN)
print(data_train.shape, data_test.shape)

Потребление памяти меньше на 40.49 Мб (-75.1%)
(47504, 119) (11877, 119)


### GradientBoostingClassifier = градиентный бустинг деревьев решений

In [4]:
%%time
model = GradientBoostingClassifier(
    max_depth=17, n_estimators=76, max_features=27, min_samples_leaf=20
).fit(data_train[columns_transformed], data_train['Response'])

CPU times: total: 2min 21s
Wall time: 2min 33s


### Предсказание данных и оценка модели

In [5]:
data_test['target'] = model.predict(data_test[columns_transformed])
print("Градиентный бустинг:", round(cohen_kappa_score(data_test["target"], data_test["Response"], weights="quadratic"), 3))

Градиентный бустинг: 0.545


Какая точность градиентного бустинга по каппа метрике с точностью до десятых?

In [8]:
print(
    "Градиентный бустинг:",
    round(
        round(cohen_kappa_score(data_test["target"], data_test["Response"], weights="quadratic"), 2), 1
    )
)

Градиентный бустинг: 0.6
